# Liminares em Habeas Corpus no Supremo Tribunal Federal
## "Liberdade, liberdade, habeas corpus sobre nós" [0] 

Importando bibliotecas e lendo csv

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns


df = pd.read_csv("../input/braziliansc-habeascorpusjan1518/BrazilianSC_HC.csv", sep=";")

Primeiras linhas

In [ ]:
df.head()

O dataset informa que a coluna Class sempre assume o valor HC, por isso vamos ignorá-la.

In [ ]:
df.drop("Class", axis=1, inplace=True)

Vamos investigar o período de tempo abragindo pelo dataset:

In [ ]:
print("data mínima: ", df.Date.min(), "data máxima: ", df.Date.max())

E listar os tipos de decisões disponíveis:

In [ ]:
df.Decision.unique()

E agora agrupar os tipos de decisões em categorias. Favoráveis são decisões que beneficiam o impetrante, ainda que parcialmente. Desfavoráveis são decisões que indeferem os pedidos dos habeas. Neutras são decisões nas quais não é produzido um juízo direto sobre a pretensão dos impetrantes, ou sobre as quais não podemos concluir que houve um exame sobre o mérito do habeas.

In [ ]:
neutras = (
    "Decisão (segredo de justiça)",
    "Prejudicado",
    "Homologada a desistência",
    "Decisão liminar (segredo de justiça)",
    "Determinado arquivamento",
    "Reconsideração",
    "Extinto o processo",
    "À Secretaria, para o regular trâmite",
    "Convertido em diligência",
    "Embargos recebidos",
    "Liminar prejudicada",
    "Declinada a competência",
    "Questão de ordem",
    "Determinada a devolução",
    "Reconsidero e julgo prejudicado o recurso interno"
)
desfavoraveis = (
    "Indeferido",
    "Negado seguimento",
    "Não conhecido(s)",
    "Liminar indeferida",
    "Não provido",
    "Denegada a ordem",
    "Embargos rejeitados",
    "Agravo regimental não conhecido",
    "Improcedente",
    "Conhecido em parte e nessa parte negado provimento",
    "Agravo regimental não provido",
    "Inadmitidos os embargos de divergência",
    "Embargos recebidos como agravo regimental desde logo não provido",
    "Embargos não conhecidos"
)
favoraveis = (
    "Liminar deferida",
    "Deferido",
    "Concedida a ordem",
    "Concedida a ordem de ofício",
    "Concedida em parte a ordem",
    "Liminar deferida em parte",
    "Deferido em parte",
    "Conhecido e provido",
    "Provido",
    "Agravo regimental provido",
    "Declarada a extinção da punibilidade",
    "Conhecido em parte e nessa parte provido"
)

Finalmente, podemos ver a distribuição de decisões favoráveis, desfavoráveis e neutras. Para os fins dessa análise, vamos nos concentrar em decisões finais e liminares, ignorando as decisões interlocutórias.

In [ ]:
df_finais = df[df["Judgment type"] != "Decisão Interlocutória"]
num_des = df_finais.Decision.isin(desfavoraveis).sum()
num_fav = df_finais.Decision.isin(favoraveis).sum()
num_neu = df_finais.Decision.isin(neutras).sum()
series_dist = pd.Series([num_des, num_fav, num_neu], index=["desfavoráveis", "favoráveis", "neutras"])

fig, axes = plt.subplots(ncols=2, figsize=(18,6))
fig.suptitle("Distribuição de decisões em habeas corpus no STF - jan/15 a jan/18", fontsize=16)
series_dist.plot.bar(ax=axes[0], rot=0)
series_dist.plot.pie(ax=axes[1], autopct="%1.0f%%")
axes[1].set_ylabel("");

Vamos nos concentrar nas decisões favoráveis. Podemos visualizar a distribuição das decisões favoráveis, e, fazendo isso, vemos que a maioria das decisões favoráveis proferidas em habeas corpus são decisões liminares.

In [ ]:
df_favoraveis = df_finais[df_finais.Decision.isin(favoraveis)]
serie_favoraveis = df_favoraveis.Decision
serie_favoraveis.value_counts().plot.bar(figsize=(12,6), grid=True, rot=30, title="Decisões favoráveis em habeas corpus no STF por tipo - jan/15 a jan/18");

In [ ]:
_df = pd.Series({
    "Mérito": (df_favoraveis["Judgment type"] == "Decisão Final").sum(),
    "Liminar": (df_favoraveis["Judgment type"] == "Decisão Liminar").sum()
})

fig, axes = plt.subplots(ncols=2, figsize=(14,6))
fig.suptitle("Decisões favoráveis em HCs no STF - mérito x liminares - jan/15 a jan/18", fontsize=16)
_df.plot.bar(ax=axes[0], rot=0)
_df.plot.pie(ax=axes[1], autopct="%1.0f%%")
axes[1].set_ylabel("");


No período analisado, como vemos, a maioria das decisões terminativas favoráveis ao impetrante se dá em sede de liminar. Sabendo disso, podemos também analisar a proporção entre liminares deferidas e indeferidas pelo Supremo nos habeas corpus:

In [ ]:
alvos = ["Liminar deferida", "Liminar indeferida"]
df_liminares = df[df["Judgment type"] == "Decisão Liminar"]
df_liminares = df_liminares[df_liminares["Decision"].isin(alvos)]
fig, axes = plt.subplots(ncols=2, figsize=(14,6))
fig.suptitle("Liminares deferidas x indeferidas no STF - em % - jan/15 a jan/18", fontsize=16)
df_liminares.Decision.value_counts().plot.bar(ax=axes[0], rot=0)
df_liminares.Decision.value_counts().plot.pie(ax=axes[1], autopct="%1.0f%%")
axes[1].set_ylabel("");

Esse tipo de análise pode auxiliar advogados como Guilherme Pompeo, que escreveu em um post no site Canal Ciências Criminais, entitulado "Existe liminar em Habeas Corpus?"[1]:

> Em breve pesquisa nos repositórios de jurisprudência, ao lançar-se as palavras chaves habeas corpus [...] e “liminar”, verifica-se que quase a totalidade dos julgados indica pelo indeferimento do pleito liminar [...]. 

> [...] cabe a todos nós, operadores do direito, verificarmos todas as situações sobre nossa responsabilidade, questionando-nos se estamos agindo em “modo automático”, utilizando aquele “modelo de petição”, ou se estamos efetivamente buscando a melhor solução dentro das nossas possibilidades para o caso concreto, seja na análise da melhor estratégia jurídica possível ou mesmo na nossa própria atuação.

De acordo com a nossa análise dos dados, embora a maioria dos pedidos de liminar em habeas corpus sejam negados pelo Supremo Tribunal Federal, a maioria das decisões favoráveis nesses processos também se dá nessa mesma sede. Considerações dessa espécie podem auxiliar advogados no processo de tomada de decisão estratégica em processos.

Por outro lado, o alto índice de decisões de concessão de liminares e de concessão de ordens de ofício indica que boa parte das decisões favoráveis proferidas em sede de habeas corpus são em casos de flagrante ilegalidade, em que é dispensada a apreciação pelo órgão colegiado, conforme o Regimento Interno do Supremo Tribunal Federal:

    Art. 192. Quando a matéria for objeto de jurisprudência consolidada do Tribunal, o Relator poderá desde logo denegar ou conceder a ordem, ainda que de ofício, à vista da documentação da petição inicial ou do teor das informações. 
    Art. 193. O Tribunal poderá, de ofício:
    ii  –  expedir ordem de habeas corpus quando, no curso de qualquer processo, verificar que alguém sofre ou se acha ameaçado de sofrer violência ou coação em sua liberdade de locomoção, por ilegalidade ou abuso de poder.
    
Podemos investigar um pouco mais essas decisões investigando se foram proferidas monocraticamente ou pelo órgão colegiado. Normalmente, é de se esperar que decisões monocráticas tratem de casos onde o grau de certeza do magistrado sobre o julgamento é elevado, seja por orientação jurisprudencial, seja por fatores internos aos processo.

In [ ]:
alvos = ["Liminar deferida", "Concedida a ordem de ofício"]
df_lim_ofi = df[df["Decision"].isin(alvos)]
col_count = df_lim_ofi[df_lim_ofi["Decision type"] == "COLEGIADA"].Decision.value_counts()
mon_count = df_lim_ofi[df_lim_ofi["Decision type"] == "MONOCRÁTICA"].Decision.value_counts()
_df = pd.DataFrame({"Monocráticas": mon_count, "Colegiadas": col_count}, index=alvos)
_df.plot.bar(figsize=(8,8), rot=0, title="Decisões monocráticas x colegiadas em concessões de liminares em hc e de hc de ofício no STF - jan/15 a jan/18");

Os dados continuam a indicar que a maior parte das decisões favoráveis em habeas corpus proferidas pelo Supremo são em casos de flagrante constrangimento ilegal, resolvidos monocraticamente por ministros. As perguntas que naturalmente surgem são: por qual razão esses casos tramitaram em todas as instâncias do judiciário brasileiro? Nenhum dos outros diversos magistrados que analisou a causa anteriormente verificou a mesma ilegalidade que o mininistro do Supremo? São esses realmente casos de flagrante ilegalidade ou são casos em que ministros utilizam de um altíssimo grau de poder discricionário? Essas questões extrapolam o escopo dos dados estudados, mas merecem ser levadas adiante como considerações.

Continuando nossa análise, podemos também analisar a distribuição de decisões liminares proferidas por cada Ministro:

In [ ]:
dic_ministros = {}
for ministro in list(df.Justice.unique()):
   dic_ministros[ministro] = df_liminares[df_liminares.Justice == ministro]

In [ ]:
cols, rows = 4, 3
fig, axes = plt.subplots(nrows=rows, ncols=cols, figsize=(18,12))

def func(pct, allvals):
    absolute = int(pct/100.*np.sum(allvals))
    return "{:d}\n{:.1f}%".format(absolute, pct)

for i, ministro in enumerate(list(df.Justice.unique())):
    a, b = divmod(i, cols)
    df_ministro = dic_ministros[ministro]
    counts = df_ministro.Decision.value_counts()
    df_ministro.Decision.value_counts().plot.pie(ax=axes[a,b], title=ministro, labels=None, autopct=lambda pct: func(pct, df_ministro.Decision.value_counts()))
    axes[a,b].set_ylabel("");

fig.suptitle("Liminares deferidas x indeferidas por ministro - jan/15 a jan/18", fontsize=16)
axes[0,3].legend(df_ministro.Decision.unique(), bbox_to_anchor=(1, 1));

Visualizando essas distribuições, é possível confirmar ou rejeitar algumas pré-concepções pessoais sobre alguns ministros e suas posições a respeito do deferimento de liminares em habeas corpus.

Referências:

* [0]: MACHADO, Nelio. Liberdade Liberdade Habeas Corpus Sobre Nós, Nélio Roberto Seidl Machado. 1994. Volume 1. Editora Impresso no Brasil.
* [1]: https://canalcienciascriminais.com.br/existe-liminar-em-habeas-corpus/